In [1]:
import matplotlib.pyplot as plt
import pandas
import numpy as np

# gerrychain imports
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from gerrychain.tree import recursive_tree_part
from gerrychain.accept import always_accept
from gerrychain.constraints import (Validator, single_flip_contiguous,
                                    within_percent_of_ideal_population, UpperBound)
from functools import partial

In [2]:
# State specific variables
state = "LA" # Louisiana
num_districts = 39 # senate districts
data_path = "./LA_data/LA_data.json"
pop_varname = "TOTPOP"
bvap_varname = "BVAP"
vap_varname = "VAP"
num_markov_steps = 50000 # in Markov chain
OPPORTUNITY_PERCENT = 0.4

In [3]:
# load the graph
graph = Graph.from_json(data_path)

In [4]:
#################################################################################
################## Get ideal population and updaters ############################
#################################################################################

def get_ideal_population(graph, pop_varname, num_districts):
    tot_pop = 0
    for i in range(len(graph.nodes)):
        tot_pop += graph.nodes[i][pop_varname]
        
    return int(tot_pop/num_districts)

ideal_population = get_ideal_population(graph, pop_varname, num_districts)

updaters = {"population": updaters.Tally(pop_varname, alias="population"),
            "bvap":updaters.Tally(bvap_varname, alias='bvap'),
            "vap":updaters.Tally(vap_varname, alias='vap'),
           }

In [5]:
####################################################################################
################ Build Initial Partition and Markov Chain ##########################
####################################################################################


## Build the initial partition

# assign each node to a district in a dict of the form {node: district}
seed_partition_dict = recursive_tree_part(graph, range(39), pop_col=pop_varname,
                                     pop_target=ideal_population,
                                     epsilon=0.01, node_repeats=1)

# use the partition dictionary to build a Partition object
seed_partition = GeographicPartition(graph, assignment=seed_partition_dict, updaters=updaters)

## Build the Markov chain
tree_proposal = partial(recom,
                       pop_col=pop_varname,
                       pop_target=ideal_population,
                       epsilon=0.05,
                       node_repeats=1
                      )

# Define the acceptance function
def increase_opportunity_districts(partition):
    opp_percent = OPPORTUNITY_PERCENT
    if partition.parent is None:
        return True
    
    curr_opp_dists = 0
    prev_opp_dists = 0
    for i in range(num_districts):
        curr_bvap_percent = partition["bvap"][i] / partition["vap"][i]
        prev_bvap_percent = partition.parent["bvap"][i] / partition.parent["vap"][i]
        
        if curr_bvap_percent >= opp_percent:
            curr_opp_dists += 1
        if prev_bvap_percent >= opp_percent:
            prev_opp_dists += 1

    return curr_opp_dists >= prev_opp_dists
    
# Constraints
popbound = within_percent_of_ideal_population(seed_partition, .1)

# Initialize the chain
recom_chain = MarkovChain(tree_proposal, 
                          Validator([popbound]), 
                          accept=increase_opportunity_districts,
                          initial_state=seed_partition, 
                          total_steps=num_markov_steps)

In [ ]:
opportunity_districts_counts = []

for partition in recom_chain:
    curr_opp_dists = 0
    for i in range(num_districts):
        bvap_percent = partition["bvap"][i] / partition["vap"][i]
        if bvap_percent >= OPPORTUNITY_PERCENT:
            curr_opp_dists += 1
    
    opportunity_districts_counts.append(curr_opp_dists)

In [ ]:
plt.plot(opportunity_districts_counts, range(num_markov_steps))
plt.title("Opportunity districts increase / remain same")
plt.xlabel("# of opportunity districts")
plt.ylabel("# of steps in chain")

# save array 
opportunity_districts_counts = np.array(opportunity_districts_counts)
np.save("opportunity_district_counts", opportunity_districts_counts)